In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['AE', 'AEs']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [4]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    if len(pmids) > 10000:
        pmids = random.choices(pmids, k=10000)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [5]:
list(zip(longforms, counts))

[('adverse events', 1461),
 ('aerobic exercise', 87),
 ('aloe emodin', 61),
 ('aqueous extract', 52),
 ('alveolar echinococcosis', 46),
 ('acute exacerbation', 46),
 ('acoustic emission', 44),
 ('angioedema', 42),
 ('atopic eczema', 37),
 ('aml1 eto', 29),
 ('and effacing', 26),
 ('acrosomal exocytosis', 20),
 ('anion exchanger', 20),
 ('acrodermatitis enteropathica', 17),
 ('acquired epilepsy', 14),
 ('autoimmune encephalitis', 13),
 ('adverse effects', 13),
 ('antiestrogens', 12),
 ('alcohol exposure', 10),
 ('aralia elata miq seem', 10),
 ('alcohol expectancies', 9),
 ('aeromedical evacuation', 8),
 ('anther extrusion', 8),
 ('axial element', 8),
 ('antiestrogen', 7),
 ('acoustoelectric', 7),
 ('androstenedione', 7),
 ('apple extract', 7),
 ('acid extract', 7),
 ('alcohol exposed', 6),
 ('arnebia euchroma', 6),
 ('aldrin epoxidase', 6),
 ('activity element', 6),
 ('absorption efficiency', 6),
 ('autoimmune epilepsy', 6),
 ('after exercise', 6),
 ('aqueous', 6),
 ('arylesterase', 6),

In [6]:
list(zip(longforms, counts))

[('adverse events', 1461),
 ('aerobic exercise', 87),
 ('aloe emodin', 61),
 ('aqueous extract', 52),
 ('alveolar echinococcosis', 46),
 ('acute exacerbation', 46),
 ('acoustic emission', 44),
 ('angioedema', 42),
 ('atopic eczema', 37),
 ('aml1 eto', 29),
 ('and effacing', 26),
 ('acrosomal exocytosis', 20),
 ('anion exchanger', 20),
 ('acrodermatitis enteropathica', 17),
 ('acquired epilepsy', 14),
 ('autoimmune encephalitis', 13),
 ('adverse effects', 13),
 ('antiestrogens', 12),
 ('alcohol exposure', 10),
 ('aralia elata miq seem', 10),
 ('alcohol expectancies', 9),
 ('aeromedical evacuation', 8),
 ('anther extrusion', 8),
 ('axial element', 8),
 ('antiestrogen', 7),
 ('acoustoelectric', 7),
 ('androstenedione', 7),
 ('apple extract', 7),
 ('acid extract', 7),
 ('alcohol exposed', 6),
 ('arnebia euchroma', 6),
 ('aldrin epoxidase', 6),
 ('activity element', 6),
 ('absorption efficiency', 6),
 ('autoimmune epilepsy', 6),
 ('after exercise', 6),
 ('aqueous', 6),
 ('arylesterase', 6),

In [12]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8891)

INFO: [2020-10-17 03:06:00] indra.ontology.bio.ontology - Loading INDRA bio ontology from cache at /home/ubuntu/.indra/bio_ontology/1.3/bio_ontology.pkl


In [13]:
result = [grounding_map, names, pos_labels]

In [14]:
result

[{'abelmoschus esculentus': 'ungrounded',
  'absence epilepsy': 'MESH:D004832',
  'absolute error': 'ungrounded',
  'absorption efficiency': 'ungrounded',
  'acetate': 'CHEBI:CHEBI:30089',
  'acetone extract': 'ungrounded',
  'acetone water extract': 'ungrounded',
  'acetylesterase': 'ungrounded',
  'acid etch': 'ungrounded',
  'acid extract': 'ungrounded',
  'acoustic emission': 'ungrounded',
  'acoustoelectric': 'ungrounded',
  'acquired epilepsy': 'MESH:D004827',
  'acridinium ester': 'ungrounded',
  'acrodermatitis enteropathica': 'MESH:C538178',
  'acrosomal exocytosis': 'ungrounded',
  'activity element': 'ungrounded',
  'activity enediyne chromophore': 'ungrounded',
  'activity expiration': 'ungrounded',
  'acute esophagitis': 'MESH:D004941',
  'acute exacerbation': 'ungrounded',
  'adapted epitopes': 'MESH:D000939',
  'adverse effects': 'NCIT:C41331',
  'adverse events': 'NCIT:C41331',
  'aerobic exercise': 'MESH:D000076663',
  'aerobic exercise training': 'ungrounded',
  'aero

In [8]:
grounding_map, names, pos_labels = [{'abelmoschus esculentus': 'ungrounded',
  'absence epilepsy': 'MESH:D004827',
  'absolute error': 'ungrounded',
  'absorption efficiency': 'ungrounded',
  'acetate': 'CHEBI:CHEBI:30089',
  'acetone extract': 'ungrounded',
  'acetone water extract': 'ungrounded',
  'acetylesterase': 'ungrounded',
  'acid etch': 'ungrounded',
  'acid extract': 'ungrounded',
  'acoustic emission': 'ungrounded',
  'acoustoelectric': 'ungrounded',
  'acquired epilepsy': 'MESH:D004827',
  'acridinium ester': 'ungrounded',
  'acrodermatitis enteropathica': 'MESH:C538178',
  'acrosomal exocytosis': 'ungrounded',
  'activity element': 'ungrounded',
  'activity enediyne chromophore': 'ungrounded',
  'activity expiration': 'ungrounded',
  'acute esophagitis': 'MESH:D004941',
  'acute exacerbation': 'ungrounded',
  'adapted epitopes': 'MESH:D000939',
  'adverse effects': 'NCIT:C41331',
  'adverse events': 'NCIT:C41331',
  'aerobic exercise': 'MESH:D000076663',
  'aerobic exercise training': 'ungrounded',
  'aeromedical evacuation': 'ungrounded',
  'aerosol': 'ungrounded',
  'african eggplant': 'ungrounded',
  'after effects': 'ungrounded',
  'after excision': 'ungrounded',
  'after exercise': 'ungrounded',
  'aftereffects': 'ungrounded',
  'afterload enhanced': 'ungrounded',
  'agronomic efficiency': 'ungrounded',
  'airway epithelium': 'MESH:D004848',
  'alcohol expectancies': 'ungrounded',
  'alcohol exposed': 'ungrounded',
  'alcohol exposure': 'ungrounded',
  'aldrin epoxidase': 'ungrounded',
  'alkylethoxylate': 'ungrounded',
  'aloe emodin': 'CHEBI:CHEBI:2607',
  'alveolar echinococcosis': 'MESH:C536591',
  'amenorrhoeic exercise': 'ungrounded',
  'aminoestrogens': 'ungrounded',
  'aml1 eto': 'MESH:C107844',
  'amla extract': 'ungrounded',
  'amniotic epithelial': 'ungrounded',
  'anaplastic ependymoma': 'MESH:D004806',
  'and effacing': 'ungrounded',
  'androstenedione': 'CHEBI:CHEBI:16422',
  'anejaculation': 'HP:HP:0012879',
  'angioedema': 'MESH:D000799',
  'angioembolization': 'ungrounded',
  'anhydrous ethanol': 'CHEBI:CHEBI:16236',
  'anion exchanger': 'ungrounded',
  'anther extrusion': 'ungrounded',
  'antiestrogen': 'CHEBI:CHEBI:50739',
  'antiestrogens': 'CHEBI:CHEBI:50739',
  'antioxidant enzymes': 'ungrounded',
  'any events': 'ungrounded',
  'aortic endarterectomy': 'MESH:D004691',
  'apple extract': 'ungrounded',
  'approximately entropy': 'ungrounded',
  'aqueous': 'ungrounded',
  'aqueous extract': 'ungrounded',
  'aralia elata miq seem': 'ungrounded',
  'arnebia euchroma': 'ungrounded',
  'arylesterase': 'ungrounded',
  'ascaris extract': 'ungrounded',
  'assimilation efficiency': 'ungrounded',
  'asthma exacerbation': 'ungrounded',
  'atopic eczema': 'MESH:D003876',
  'atrial endocardium': 'MESH:D004699',
  'attaching effacing': 'ungrounded',
  'auto encoder': 'ungrounded',
  'autoencoder': 'ungrounded',
  'autoimmune encephalitis': 'MESH:C535841',
  'autoimmune epilepsy': 'MESH:D004827',
  'axial element': 'GO:GO:0000800',
  'axillaris × p exserta': 'ungrounded',
  'serious adverse effects': 'ungrounded'},
 {'CHEBI:CHEBI:30089': 'acetate',
  'MESH:D004827': 'Epilepsy',
  'MESH:C538178': 'Acrodermatitis enteropathica',
  'MESH:D004941': 'Esophagitis',
  'MESH:D000939': 'Epitopes',
  'NCIT:C41331': 'Adverse Event',
  'MESH:D000076663': 'Endurance Training',
  'MESH:D004848': 'Epithelium',
  'CHEBI:CHEBI:2607': 'Aloe emodin',
  'MESH:C536591': 'Alveolar echinococcosis',
  'MESH:C107844': 'AML1-ETO fusion protein, human',
  'MESH:D004806': 'Ependymoma',
  'CHEBI:CHEBI:16422': 'androst-4-ene-3,17-dione',
  'HP:HP:0012879': 'Anejaculation',
  'MESH:D000799': 'Angioedema',
  'CHEBI:CHEBI:16236': 'ethanol',
  'CHEBI:CHEBI:50739': 'estrogen receptor modulator',
  'MESH:D004691': 'Endarterectomy',
  'MESH:D003876': 'Dermatitis, Atopic',
  'MESH:D004699': 'Endocardium',
  'MESH:C535841': "Hashimoto's encephalitis",
  'GO:GO:0000800': 'lateral element'},
 ['CHEBI:CHEBI:2607',
  'CHEBI:CHEBI:50739',
  'MESH:C107844',
  'MESH:C536591',
  'MESH:D000076663',
  'MESH:D000799',
  'MESH:C538178',
  'NCIT:C41331',
  'GO:GO:0000800']]

In [9]:
excluded_longforms = []

In [10]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [11]:
additional_entities = {}

In [12]:
unambiguous_agent_texts = {}

In [13]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [15]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [16]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [17]:
intersection1

[('HGNC:17981', 'HGNC:17981', 29)]

In [18]:
intersection2

[('ungrounded', 'HGNC:17981', 0)]

In [20]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items()])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=['RTCA', 'RTCD1', 'RPC', 'RTC1', 'RTC'])
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items()])

In [ ]:
names.update(additional_entitie)

In [14]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-10-22 04:41:53] /adeft/PythonRepos/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-10-22 04:42:46] /adeft/PythonRepos/adeft/adeft/modeling/classify.py - Best f1 score of 0.9508841182756207 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [15]:
classifier.stats

{'label_distribution': {'ungrounded': 292,
  'CHEBI:CHEBI:16422': 6,
  'CHEBI:CHEBI:50739': 19,
  'MESH:C538178': 14,
  'NCIT:C41331': 1339,
  'MESH:D000076663': 54,
  'MESH:D003876': 26,
  'MESH:C536591': 29,
  'MESH:D000799': 32,
  'GO:GO:0000800': 16,
  'CHEBI:CHEBI:2607': 52,
  'MESH:C107844': 20,
  'MESH:D004941': 4,
  'CHEBI:CHEBI:16236': 2,
  'MESH:D004827': 15,
  'MESH:D004848': 3,
  'CHEBI:CHEBI:30089': 2,
  'MESH:D004806': 2,
  'MESH:D000939': 2,
  'HP:HP:0012879': 1,
  'MESH:C535841': 8,
  'MESH:D004691': 1,
  'MESH:D004699': 1},
 'f1': {'mean': 0.950884, 'std': 0.006709},
 'precision': {'mean': 0.946788, 'std': 0.006139},
 'recall': {'mean': 0.960012, 'std': 0.011241},
 'MESH:C535841': {'f1': {'mean': 0.466667, 'std': 0.4},
  'pr': {'mean': 0.4, 'std': 0.374166},
  'rc': {'mean': 0.6, 'std': 0.489898}},
 'MESH:C538178': {'f1': {'mean': 0.88, 'std': 0.09798},
  'pr': {'mean': 0.8, 'std': 0.163299},
  'rc': {'mean': 1.0, 'std': 0.0}},
 'MESH:D003876': {'f1': {'mean': 0.484127

In [17]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [18]:
disamb.dump(model_name, results_path)

In [20]:
print(disamb.info())

Disambiguation model for AE, and AEs

Produces the disambiguations:
	AML1-ETO fusion protein, human*	MESH:C107844
	Acrodermatitis enteropathica*	MESH:C538178
	Adverse Event*	NCIT:C41331
	Aloe emodin*	CHEBI:CHEBI:2607
	Alveolar echinococcosis*	MESH:C536591
	Anejaculation	HP:HP:0012879
	Angioedema*	MESH:D000799
	Dermatitis, Atopic	MESH:D003876
	Endarterectomy	MESH:D004691
	Endocardium	MESH:D004699
	Endurance Training*	MESH:D000076663
	Ependymoma	MESH:D004806
	Epilepsy	MESH:D004827
	Epithelium	MESH:D004848
	Epitopes	MESH:D000939
	Esophagitis	MESH:D004941
	Hashimoto's encephalitis	MESH:C535841
	acetate	CHEBI:CHEBI:30089
	androst-4-ene-3,17-dione	CHEBI:CHEBI:16422
	estrogen receptor modulator*	CHEBI:CHEBI:50739
	ethanol	CHEBI:CHEBI:16236
	lateral element*	GO:GO:0000800

Class level metrics:
--------------------
Grounding                     	Count	F1     
                 Adverse Event*	1339	0.97185
                    Ungrounded	 292	0.83378
            Endurance Training*	  54	0.76578
   

In [21]:
model_to_s3(disamb)